# This notebook is mainly used for the capstone project

In [ ]:
import pandas as pd


In [ ]:
import numpy as np

In [41]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# All the 3 parts of the project are done in a single notebook.

# 1. Segmenting and Clustering Neighborhoods in Toronto

In [7]:
import pandas as pd
import numpy as np 
from pandas.io.json import json_normalize
import json
# Used https://beautiful-soup-4.readthedocs.io/en/latest/
#importing soup libraries
from bs4 import BeautifulSoup
import requests




# Extracting data from Wikipedia page

In [8]:
wiki_url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_result= requests.get(wiki_url).text
soup = BeautifulSoup(url_result,"lxml")
#print(soup.prettify())
 

In [9]:
#Only process the cells that have an assigned borough
#Taken help from https://stackoverflow.com/questions/23377533/python-beautifulsoup-parsing-table
wiki_table= soup.find("table",class_="wikitable")
Nhood_rows=wiki_table.find_all("tr")

#Trying to extract the column names(Postcode,Borough,Neighbourhood)
Nhood=[]
for i in Nhood_rows:
    #deletes the first and last items with empty strings
    res=i.text.split("\n")[1:-1]
    Nhood.append(res)

Nhood[0:11]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned']]

# Converting the data to Pandas dataframe.

In [10]:
Nhood[0][-1]="Neighborhood"
df_Nhood=pd.DataFrame(Nhood[1:],columns=Nhood[0])
#Prints Borough and Neighborhood 'Not assigned' values too
df_Nhood.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
#Removing 'Not assigned' from Borough and Neighborhood
borough_notassigned=df_Nhood.index[df_Nhood["Borough"]=="Not assigned"]
nhood__notassigned=df_Nhood.index[df_Nhood["Neighborhood"]=="Not assigned"]
BoroughNhood_notassigned=borough_notassigned & nhood__notassigned

#This only process the cells that have an assigned borough. We ignore cells with a borough that is Not assigned.
df_Nhood.drop(df_Nhood.index[borough_notassigned],inplace=True)
df_Nhood.reset_index(drop=True,inplace=True)
df_Nhood.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [12]:
nhood__notassigned=df_Nhood.index[df_Nhood["Neighborhood"]=="Not assigned"]
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for j in nhood__notassigned:
    df_Nhood["Neighborhood"][j]=df_Nhood["Borough"][j]

df_Nhood.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [13]:
#Grouping the neighbourhoods with the same postal code. Separated by comma.
postalCode_group=df_Nhood.groupby("Postcode")
Nhood_group=postalCode_group['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
borough_group=postalCode_group['Borough'].apply(lambda x: set(x).pop())
df_group= pd.DataFrame(list(zip(borough_group.index,borough_group,Nhood_group)))
df_group.columns=["Postcode", "Borough", "Neighborhood"]
df_group.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
#using the .shape method to print the number of rows of your dataframe
df_group.shape

(103, 3)

# 2. The latitude and the longitude coordinates of each neighborhood.

In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
#Body is the Geospatial_Coordinates.csv
df_coordinates = pd.read_csv(body)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [75]:
postcode_coordinates=df_group.join(df_coordinates.set_index("Postal Code"),on="Postcode")
postcode_coordinates.head(15)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# 3. Explore and cluster the neighborhoods in Toronto.

In [76]:
from sklearn.cluster import KMeans 
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes
import folium 

In [77]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
GeoLocator = Nominatim(user_agent='My-IBMNotebook')

In [21]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Coordinates of Toronto are: {}, {}.".format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Coordinates of Toronto are: 43.653963, -79.387207.


In [73]:
toronto_map=folium.Map(location=[latitude, longitude], zoom_start=11)

for location in postcode_coordinates.itertuples():
    label = 'Postal Code: {};  Borough: {};  Neighborhoods: {}'.format(location[1], location[2], location[3])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [location[-2], location[-1]],
        radius=1,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map) 
    folium.Circle(
        radius=500,
        popup=label,
        location=[location[-2], location[-1]],
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(toronto_map) 
    
toronto_map
        
 

Reducing the radius of the neighborhood of toronto.

In [23]:
toronto_data = postcode_coordinates[postcode_coordinates['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(38, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [74]:
toronto_map=folium.Map(location=[latitude, longitude], zoom_start=11)
for lat,long,label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

We will be using Foursquare API to explore and segment neighbourhoods in Toronto.


In [46]:
CLIENT_ID="1SYPLBVQSBJROI3EAS5NGXGYBQ0WZ0JFUURTLJZYQL4TACGD"
CLIENT_SECRET="JEWQ3X5RESQT02ZCCGUSTVA4BFAZ3MKKJ1TIKT31TPHMTODI"
VERSION="20190803"
LIMIT = 30
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: 1SYPLBVQSBJROI3EAS5NGXGYBQ0WZ0JFUURTLJZYQL4TACGD
CLIENT_SECRET:JEWQ3X5RESQT02ZCCGUSTVA4BFAZ3MKKJ1TIKT31TPHMTODI


In [49]:
#Different venues as a dataframe
def get_venues(names,latitudes,longitudes,radius=500):
    list_venue=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #get request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        list_venue.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in list_venue for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [50]:
postcode_coordinates=toronto_data
toronto_venue=get_venues(names=postcode_coordinates['Neighborhood'],latitudes=postcode_coordinates['Latitude'],longitudes=postcode_coordinates['Longitude'])





The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [58]:
#print(toronto_venue.shape)
toronto_venue.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Count of venues for Borough

In [60]:
toronto_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30
Christie,16,16,16,16,16,16


Unique categories for all the returned venues.

In [61]:
print('{} uniques categories.'.format(len(toronto_venue['Venue Category'].unique())))

195 uniques categories.


Analyzing each Borough Neighbourhood

In [62]:
toronto_onehot = pd.get_dummies(toronto_venue[['Venue Category']], prefix="", prefix_sep="")

# we are adding neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venue['Neighborhood'] 

# we are moving neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping rows by neighbourhood and taking the mean of the frequency of occurrence of each category.

In [63]:
group_toronto= toronto_onehot.groupby('Neighborhood').mean().reset_index()
group_toronto.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


Now, we are going to put the grouped data into a Pandas dataframe.

In [64]:
num_top_venues = 5
for x in group_toronto['Neighborhood']:
    print("----"+x+"----")
    temp =group_toronto[group_toronto['Neighborhood'] == x].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0        Steakhouse  0.10
1       Pizza Place  0.07
2             Hotel  0.07
3       Coffee Shop  0.07
4  Asian Restaurant  0.07


----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1        Cocktail Bar  0.07
2                Café  0.07
3      Farmers Market  0.07
4            Beer Bar  0.07


----Brockton, Exhibition Place, Parkdale Village----
                  venue  freq
0        Breakfast Spot  0.09
1                  Café  0.09
2           Coffee Shop  0.09
3            Restaurant  0.04
4  Caribbean Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Garden Center  0.06
3                Park  0.06
4          Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Ser

In [65]:
#We are sorting the venues in descending order
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# we are creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# creating a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = group_toronto['Neighborhood']

for ind in np.arange(group_toronto.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] =most_common_venues(group_toronto.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(38, 11)

# Clustering the Neighbourhood.

We are going to use K-means to cluster the neighbourhood into 6 clusters.

In [68]:
# we are setting the number of clusters to 10.
k_clusters = 10

group_toronto_clustering = group_toronto.drop('Neighborhood', 1)
# running k-means clustering
k_means = KMeans(n_clusters=k_clusters, random_state=1).fit(group_toronto_clustering)

# checking the cluster labels generated for each row in the dataframe
print(k_means.labels_[0:10])
print(len(k_means.labels_))

[1 1 2 1 1 2 2 1 2 1]
38


In [69]:
postcode_coordinates.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [71]:
merge_toronto=postcode_coordinates

#we are adding the clustering labels
merge_toronto['Cluster Labels'] = k_means.labels_

#we are merging group_toronto with toronto_data to add the latitude/longitude for each neighbourhood
merge_toronto = merge_toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merge_toronto.tail()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,2,Coffee Shop,Café,Breakfast Spot,Gym,Intersection,Performing Arts Venue,Pet Store,Climbing Gym,Convenience Store,Caribbean Restaurant
34,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,2,Café,Bar,Mexican Restaurant,Music Venue,Italian Restaurant,Bakery,Cajun / Creole Restaurant,Flea Market,Fast Food Restaurant,Arts & Crafts Store
35,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,9,Gift Shop,Breakfast Spot,Coffee Shop,Movie Theater,Eastern European Restaurant,Dog Run,Bar,Bank,Dessert Shop,Restaurant
36,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,1,Pizza Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Falafel Restaurant,Sandwich Place,Bookstore,Latin American Restaurant,Dessert Shop
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,1,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Butcher,Burrito Place,Restaurant,Brewery,Skate Park


Visualizing the clusters.

In [72]:
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=11)

#colours for the clusters are set.
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merge_toronto['Latitude'], merge_toronto['Longitude'], merge_toronto['Neighborhood'],k_means.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)
       
cluster_map